In [2]:
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [3]:
data_folder = Path("data/predict-energy-behavior-of-prosumers")

files = {
    "train": "train.csv",
    "gas_prices": "gas_prices.csv",
    "client": "client.csv",
    "electricity_prices": "electricity_prices.csv",
    "forecast_weather": "forecast_weather.csv",
    "historical_weather": "historical_weather.csv",
}

In [6]:
train_df = pl.read_csv(
    data_folder / files["train"]
)

gas_df = pl.read_csv(
    data_folder / files["gas_prices"]
)

client_df = pl.read_csv(
    data_folder / files["client"]
)

electricity_df = pl.read_csv(
    data_folder / files["electricity_prices"]
)

forecast_df = pl.read_csv(
    data_folder / files["forecast_weather"]
)

historical_df = pl.read_csv(
    data_folder / files["historical_weather"]
)

In [7]:
train_df.describe()

statistic,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
str,f64,f64,f64,f64,f64,str,f64,f64,f64
"""count""",2.018352e6,2.018352e6,2.018352e6,2.017824e6,2.018352e6,"""2018352""",2.018352e6,2.018352e6,2.018352e6
"""null_count""",0.0,0.0,0.0,528.0,0.0,"""0""",0.0,0.0,0.0
"""mean""",7.297034,0.536826,1.898927,274.85556,0.5,null,321.874599,1009175.5,33.045376
"""std""",4.78099,0.498642,1.081766,909.502378,0.5,null,182.634314,582648.179597,19.590594
"""min""",0.0,0.0,0.0,0.0,0.0,"""2021-09-01 00:00:00""",0.0,0.0,0.0
"""25%""",3.0,0.0,1.0,0.378,0.0,null,166.0,504588.0,16.0
"""50%""",7.0,1.0,2.0,31.133,1.0,null,323.0,1.009176e6,33.0
"""75%""",11.0,1.0,3.0,180.206,1.0,null,479.0,1.513763e6,50.0
"""max""",15.0,1.0,3.0,15480.274,1.0,"""2023-05-31 23:00:00""",637.0,2.018351e6,68.0


In [8]:
gas_df.describe()

statistic,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
str,str,f64,f64,str,f64
"""count""","""637""",637.0,637.0,"""637""",637.0
"""null_count""","""0""",0.0,0.0,"""0""",0.0
"""mean""",null,95.03675,107.754631,null,319.0
"""std""",null,47.552295,54.743666,null,184.030342
"""min""","""2021-09-01""",28.1,34.0,"""2021-08-31""",1.0
"""25%""",null,60.0,67.53,null,160.0
"""50%""",null,85.21,93.47,null,319.0
"""75%""",null,109.0,130.74,null,478.0
"""max""","""2023-05-30""",250.0,305.0,"""2023-05-29""",637.0


In [9]:
client_df.describe()

statistic,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
str,f64,f64,f64,f64,f64,str,f64
"""count""",41919.0,41919.0,41919.0,41919.0,41919.0,"""41919""",41919.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,"""0""",0.0
"""mean""",1.898996,7.297097,73.345118,1450.771451,0.536773,null,322.898876
"""std""",1.081713,4.78075,144.064389,2422.23312,0.498652,null,182.075724
"""min""",0.0,0.0,5.0,5.5,0.0,"""2021-09-01""",2.0
"""25%""",1.0,3.0,13.0,321.9,0.0,null,167.0
"""50%""",2.0,7.0,32.0,645.2,1.0,null,324.0
"""75%""",3.0,11.0,70.0,1567.15,1.0,null,480.0
"""max""",3.0,15.0,1517.0,19314.31,1.0,"""2023-05-29""",637.0


In [10]:
electricity_df.describe()

statistic,forecast_date,euros_per_mwh,origin_date,data_block_id
str,str,f64,str,f64
"""count""","""15286""",15286.0,"""15286""",15286.0
"""null_count""","""0""",0.0,"""0""",0.0
"""mean""",null,157.064176,null,318.99071
"""std""",null,121.148625,null,183.890301
"""min""","""2021-09-01 00:00:00""",-10.06,"""2021-08-31 00:00:00""",1.0
"""25%""",null,85.29,null,160.0
"""50%""",null,128.28,null,319.0
"""75%""",null,199.8,null,478.0
"""max""","""2023-05-30 23:00:00""",4000.0,"""2023-05-29 23:00:00""",637.0


In [11]:
forecast_df.describe()

statistic,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64
"""count""",3.424512e6,3.424512e6,"""3424512""",3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,"""3424512""",3.424512e6,3.42451e6,3.424512e6,3.424512e6
"""null_count""",0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,2.0,0.0,0.0
"""mean""",58.65,24.95,null,24.5,5.743913,2.411946,0.394665,0.434645,0.359069,0.681993,1.255446,0.725011,319.0,null,151.188212,110.764207,0.000025,0.000079
"""std""",0.687386,2.015565,null,13.853401,7.844206,7.121432,0.444043,0.438635,0.420156,0.400963,3.9953,4.223752,183.885861,null,256.506903,187.444373,0.000122,0.000278
"""min""",57.6,21.7,"""2021-09-01 02:00:00""",1.0,-27.499396,-29.683569,0.0,0.0,0.0,0.0,-17.577179,-22.116119,1.0,"""2021-09-01 03:00:00""",-0.773333,-0.325833,-0.000004,-0.000015
"""25%""",58.2,23.2,null,13.0,0.260645,-2.364355,0.0,0.000336,0.0,0.264893,-1.46669,-1.978104,160.0,null,0.0,0.0,0.0,0.0
"""50%""",58.8,25.2,null,25.0,4.872705,1.835596,0.088669,0.230546,0.101196,0.977267,1.468682,0.943322,319.0,null,0.0,0.604444,0.0,0.0
"""75%""",59.1,26.7,null,36.0,11.146387,7.302026,0.978485,0.999459,0.900085,1.0,3.813531,3.507628,478.0,null,212.844444,144.172326,0.0,0.000028
"""max""",59.7,28.2,"""2023-05-30 02:00:00""",48.0,31.810693,23.680566,1.000008,1.000008,1.000008,1.000008,22.573198,19.314369,637.0,"""2023-06-01 02:00:00""",954.422222,848.714444,0.004833,0.016516


In [13]:
historical_df.describe()

statistic,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""1710802""",1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,5.740968,2.240312,0.04962,0.016049,1009.281515,60.912696,46.685927,34.40698,36.051408,4.849871,197.869419,106.490504,64.452917,42.037587,58.649999,24.949999,319.270778
"""std""",null,8.025647,7.224357,0.207911,0.074629,13.088915,37.769048,40.747598,38.327693,41.358521,2.47545,89.937978,179.944912,133.409951,61.952251,0.687387,2.015564,183.729798
"""min""","""2021-09-01 00:00:00""",-23.7,-25.9,0.0,0.0,942.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.6,21.7,1.0
"""25%""",null,0.0,-2.6,0.0,0.0,1001.5,25.0,3.0,0.0,0.0,3.0,139.0,0.0,0.0,0.0,57.9,23.2,160.0
"""50%""",null,5.1,1.7,0.0,0.0,1010.4,72.0,39.0,16.0,10.0,4.5,208.0,1.0,0.0,1.0,58.5,24.7,319.0
"""75%""",null,11.2,7.2,0.0,0.0,1018.0,100.0,94.0,72.0,85.0,6.277778,263.0,140.0,47.0,74.0,59.1,26.7,478.0
"""max""","""2023-05-30 10:00:00""",32.6,23.8,16.8,2.66,1049.3,100.0,100.0,100.0,100.0,21.75,360.0,849.0,754.0,388.0,59.7,28.2,637.0


In [16]:
train_df.shape

(2018352, 9)

In [17]:
gas_df.shape

(637, 5)

In [18]:
client_df.shape

(41919, 7)

In [19]:
electricity_df.shape

(15286, 4)

In [20]:
forecast_df.shape

(3424512, 18)

In [21]:
historical_df.shape

(1710802, 18)